<a href="https://colab.research.google.com/github/kcalizadeh/PDP_data_processing/blob/master/w2v_for_imports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports and Mounting Drive

In [1]:
# this cell mounts drive, sets the correct directory, then imports all functions
# and relevant libraries via the functions.py file
from google.colab import drive
import sys

drive.mount('/gdrive',force_remount=True)

drive_path = '/gdrive/MyDrive/Colab_Projects/Phil_NLP'

sys.path.append(drive_path)

Mounted at /gdrive


In [26]:
import pandas as pd
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from gensim.models import Phrases
from gensim.models.phrases import Phraser


# a function for quickly testing w2v models
def test_w2v(model, pairs):
  for (pos, neg) in pairs:
    math_result = model.most_similar(positive=pos, negative=neg)
    print(f'Positive - {pos}\tNegative - {neg}')
    [print(f"- {result[0]} ({round(result[1],5)})") for result in math_result[:5]]
    print()

### Load the Data

In [4]:
df = pd.read_csv('/gdrive/MyDrive/Colab_Projects/philosophy_data_project/phil_nlp.csv')

df.sample(5)

,title,author,school,sentence_spacy,sentence_str,sentence_length,sentence_lowered,tokenized_txt,lemmatized_str
78501,Aristotle - Complete Works,Aristotle,aristotle,And those who are in a position of superiority...,And those who are in a position of superiority...,188,and those who are in a position of superiority...,"['and', 'those', 'who', 'are', 'in', 'position...",and those who be in a position of superiority...
178416,Philosophical Troubles,Kripke,analytic,"This shows that, normally, if the third condit...","This shows that, normally, if the third condit...",74,"this shows that, normally, if the third condit...","['this', 'shows', 'that', 'normally', 'if', 't...","this show that , normally , if the third cond..."
283301,Elements Of The Philosophy Of Right,Hegel,german_idealism,"that is, Ideas in general, and hence also the ...","that is, Ideas in general, and hence also the ...",234,"that is, ideas in general, and hence also the ...","['that', 'is', 'ideas', 'in', 'general', 'and'...","that is , Ideas in general , and hence also t..."
65488,Aristotle - Complete Works,Aristotle,aristotle,"Now their bite is not rough and fiery, but mal...","Now their bite is not rough and fiery, but mal...",53,"now their bite is not rough and fiery, but mal...","['now', 'their', 'bite', 'is', 'not', 'rough',...","now -PRON- bite be not rough and fiery , but ..."
85882,Aristotle - Complete Works,Aristotle,aristotle,In addition stringent laws must be laid down f...,In addition stringent laws must be laid down f...,232,in addition stringent laws must be laid down f...,"['in', 'addition', 'stringent', 'laws', 'must'...",in addition stringent law must be lay down fo...


In [5]:
# using gensim's built-in tokenizer 
df['gensim_tokenized'] = df['sentence_str'].map(lambda x: simple_preprocess(x.lower(),deacc=True,
                                                        max_len=100))

In [6]:
# check how it worked
print(df.iloc[290646]['sentence_str'])
df['gensim_tokenized'][290646]

A spider conducts operations that resemble those of a weaver, and a bee puts to shame many an architect in the construction of her cells.


['spider',
 'conducts',
 'operations',
 'that',
 'resemble',
 'those',
 'of',
 'weaver',
 'and',
 'bee',
 'puts',
 'to',
 'shame',
 'many',
 'an',
 'architect',
 'in',
 'the',
 'construction',
 'of',
 'her',
 'cells']

Hmm, an interesting observation. 

### Transfer Learning with GloVe

We'll import GloVe vectors as w2v, then use those as a base from which to train new vectors that are tuned to our corpus.

In [27]:
# load the vectors. other vector sizes were used but yielded generally less sensible models
glove_file = datapath('/gdrive/MyDrive/Colab_Projects/philosophy_data_project/glove.6B.50d.txt')
tmp_file = get_tmpfile("test_word2vec.txt")

_ = glove2word2vec(glove_file, tmp_file)

glove_vectors = KeyedVectors.load_word2vec_format(tmp_file)

In [28]:
pairs_to_try = [(['law', 'moral'], []),
                (['self', 'consciousness'], []),
                (['dialectic'], []),
                (['logic'], []),
]

In [29]:
# check out how GloVe works on our test pairs
test_w2v(glove_vectors, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- morality (0.82654)
- legal (0.82652)
- laws (0.81529)
- constitutional (0.80616)
- fundamental (0.80217)

Positive - ['self', 'consciousness']	Negative - []
- sense (0.83446)
- mind (0.79755)
- vision (0.78202)
- belief (0.78031)
- life (0.77984)

Positive - ['dialectic']	Negative - []
- hegelian (0.88376)
- dialectical (0.83417)
- dialectics (0.80672)
- materialist (0.77674)
- metaphysics (0.77488)

Positive - ['logic']	Negative - []
- reasoning (0.81405)
- intuitionistic (0.76531)
- concepts (0.75831)
- logical (0.75604)
- theory (0.75026)



Now we want these to be trained on our actual philosophical texts - that way we can see how different thinkers use different words and potentially use the vectors for classification.

So in the cells below we train the existing GloVe model on on the German Idealist texts as a test.

#### German Idealism Example

In [30]:
# isolate the relevant school
documents = df[df['school'] == 'german_idealism']['gensim_tokenized']

# format the series to be used
stopwords = []

sentences = [sentence for sentence in documents]
cleaned = []
for sentence in sentences:
  cleaned_sentence = [word.lower() for word in sentence]
  cleaned_sentence = [word for word in sentence if word not in stopwords]
  cleaned.append(cleaned_sentence)

# get bigrams
bigram = Phrases(cleaned, min_count=20, threshold=10, delimiter=b' ')
bigram_phraser = Phraser(bigram)

bigramed_tokens = []
for sent in cleaned:
    tokens = bigram_phraser[sent]
    bigramed_tokens.append(tokens)

# run again to get trigrams
trigram = Phrases(bigramed_tokens, min_count=20, threshold=10, delimiter=b' ')
trigram_phraser = Phraser(trigram)

trigramed_tokens = []
for sent in bigramed_tokens:
    tokens = trigram_phraser[sent]
    trigramed_tokens.append(tokens)

# build a toy model to update with
base_model = Word2Vec(size=300, min_count=5)
base_model.build_vocab(trigramed_tokens)
total_examples = base_model.corpus_count

# add GloVe's vocabulary & weights
base_model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

# train on our data
base_model.train(trigramed_tokens, total_examples=total_examples, epochs=base_model.epochs)
base_model_wv = base_model.wv
del base_model

In [31]:
test_w2v(base_model_wv, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- freedom (0.82856)
- rule (0.80684)
- morality (0.80089)
- moral law (0.7894)
- highest good (0.78798)

Positive - ['self', 'consciousness']	Negative - []
- self consciousness (0.91867)
- essence (0.86718)
- objectivity (0.86468)
- negativity (0.86383)
- positedness (0.86014)

Positive - ['dialectic']	Negative - []
- deduction (0.90418)
- antinomy (0.89887)
- exposition (0.89578)
- method (0.8956)
- definition (0.89019)

Positive - ['logic']	Negative - []
- science (0.84214)
- pure reason (0.83874)
- idealism (0.83832)
- doctrine (0.83681)
- metaphysics (0.81759)



These seem to reflect the true uses of words in German Idealism. 'Self' + 'consciousness' is rightly associated with 'self consciousness' and 'moral' + 'law' with 'moral law'. It even identifies the German Idealist tendency to unify logic and metaphysics. 

These vectors can be fairly said to reflect how german idealists use these terms. Moreover, they are significantly different than the original GloVe model, which indicates that there was real learning going on here.

For comparison, let's check these same terms, but as used by Phenomenologists.

#### Phenomenology Comparision

In [36]:
def train_glove(source_type, source, glove_vectors, threshold=10, stopwords=[],
                min_count=20):
  # isolate the relevant school
  documents = df[df[source_type] == source]['gensim_tokenized']

  # format the series to be used
  stopwords = []

  sentences = [sentence for sentence in documents]
  cleaned = []
  for sentence in sentences:
    cleaned_sentence = [word.lower() for word in sentence]
    cleaned_sentence = [word for word in sentence if word not in stopwords]
    cleaned.append(cleaned_sentence)

  # get bigrams
  bigram = Phrases(cleaned, min_count=min_count, threshold=threshold, 
                   delimiter=b' ')
  bigram_phraser = Phraser(bigram)

  bigramed_tokens = []
  for sent in cleaned:
      tokens = bigram_phraser[sent]
      bigramed_tokens.append(tokens)

  # run again to get trigrams
  trigram = Phrases(bigramed_tokens, min_count=min_count, threshold=threshold, 
                    delimiter=b' ')
  trigram_phraser = Phraser(trigram)

  trigramed_tokens = []
  for sent in bigramed_tokens:
      tokens = trigram_phraser[sent]
      trigramed_tokens.append(tokens)

  # build a toy model to update with
  model = Word2Vec(size=300, min_count=5)
  model.build_vocab(trigramed_tokens)
  total_examples = model.corpus_count

  # add GloVe's vocabulary & weights
  model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

  # train on our data
  model.train(trigramed_tokens, total_examples=total_examples, epochs=model.epochs)
  model_wv = model.wv
  del model
  return model_wv

In [37]:
ph_model = train_glove(source_type='school', source='phenomenology', glove_vectors=glove_vectors)

In [34]:
test_w2v(ph_model, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- magical (0.9949)
- nervous (0.99348)
- exhibitings (0.99332)
- gestures (0.99329)
- categorial (0.99328)

Positive - ['self', 'consciousness']	Negative - []
- potentiality (0.95779)
- nature (0.94881)
- authentic (0.93771)
- existence (0.93201)
- future (0.93043)

Positive - ['dialectic']	Negative - []
- contact (0.99394)
- steps (0.99185)
- journey (0.99121)
- purposes (0.9898)
- placing (0.9894)

Positive - ['logic']	Negative - []
- definition (0.97844)
- phenomenology (0.97734)
- infinity (0.97683)
- constancy (0.97512)
- modification (0.97508)



Using the phenomenology vectors on some central terms of German idealism once again yields some pretty compelling results, except for where the words are rarely used by the phenomenologists. This is to be expected. Let's try the word vectors on some central terms of phenomenology.

In [35]:
pairs_to_try = [(['perception'], []),
                (['dasein'], []),
                (['consciousness'], []),
                (['method'], []),]

test_w2v(ph_model, pairs_to_try)

Positive - ['perception']	Negative - []
- act (0.91805)
- reality (0.91235)
- knowledge (0.91063)
- certainty (0.90632)
- care (0.90189)

Positive - ['dasein']	Negative - []
- itself (0.88474)
- being (0.86611)
- truth (0.84091)
- consciousness (0.83831)
- future (0.8306)

Positive - ['consciousness']	Negative - []
- future (0.92622)
- death (0.91877)
- nature (0.90971)
- potentiality (0.90915)
- truth (0.90738)

Positive - ['method']	Negative - []
- spirit (0.97481)
- necessity (0.97436)
- ontology (0.97354)
- origin (0.9642)
- process (0.95573)



These look pretty strong. Overall, the GloVe-traiend vectors seem to be an effective tool for revealing how a word is used by a school. 

#### Training on every school & author

To further explore this, we'll train w2v models in this way for each school and examine how each of them looks at the same word - 'philosophy.' We can use these in our future dashboard work.

In [ ]:
w2v_dict = {}

for school in df['school'].unique():
  w2v_dict[school] = train_glove(school, glove_vectors=glove_vectors)
  print(f'{school} completed')

plato completed
aristotle completed
empiricism completed
rationalism completed
analytic completed
continental completed
phenomenology completed
german_idealism completed
communism completed
capitalism completed


In [ ]:
for school in df['school'].unique():
  print(f'\t{school.upper()}')
  print('----------------------')
  test_w2v(w2v_dict[school], [(['philosophy'], [])])

	PLATO
----------------------
Positive - ['philosophy']	Negative - []
- relief (0.94646)
- springs (0.94496)
- friendship (0.94237)
- greece (0.93422)
- regime (0.93311)

	ARISTOTLE
----------------------
Positive - ['philosophy']	Negative - []
- respiration (0.91699)
- oratory (0.87146)
- shrillness (0.87085)
- holders (0.86975)
- memory (0.85944)

	EMPIRICISM
----------------------
Positive - ['philosophy']	Negative - []
- religion (0.93495)
- mankind (0.92261)
- doctrine (0.92259)
- inquiry (0.9155)
- faith (0.90897)

	RATIONALISM
----------------------
Positive - ['philosophy']	Negative - []
- person (0.95782)
- return (0.92869)
- death (0.92862)
- fall (0.92596)
- public (0.9258)

	ANALYTIC
----------------------
Positive - ['philosophy']	Negative - []
- philosophical (0.90815)
- hahn (0.85215)
- carnap (0.84692)
- semantics (0.84195)
- davidson (0.83312)

	CONTINENTAL
----------------------
Positive - ['philosophy']	Negative - []
- metaphysics (0.9689)
- unreason (0.95789)
- hist

Interestingly, many of these top words align quite strongly with the school's general attitude towards philosophy. Continental thinkers mentioning unreason, analytic philosophers focusing on semantics, and phenomenologists associating philosophy with a method all track well. The ones that don't make sense are those that don't problematize the nature of philosophy to any great degree - capitalist thinkers aren't out there trying to discuss the nature of philosophy.

We'd also like vectors trained for each individual author. We can use these in our dashboard to enable intra-school comparisons of authors and generally allow for more fine-grained data exploration.

In [ ]:
#@title Glove Training Function Modified for Authors
def train_glove_author(school, glove_vectors, threshold=10, stopwords=[],
                min_count=20):
  # isolate the relevant school
  documents = df[df['author'] ==school]['gensim_tokenized']

  # format the series to be used
  stopwords = []

  sentences = [sentence for sentence in documents]
  cleaned = []
  for sentence in sentences:
    cleaned_sentence = [word.lower() for word in sentence]
    cleaned_sentence = [word for word in sentence if word not in stopwords]
    cleaned.append(cleaned_sentence)

  # get bigrams
  bigram = Phrases(cleaned, min_count=min_count, threshold=threshold, 
                   delimiter=b' ')
  bigram_phraser = Phraser(bigram)

  bigramed_tokens = []
  for sent in cleaned:
      tokens = bigram_phraser[sent]
      bigramed_tokens.append(tokens)

  # run again to get trigrams
  trigram = Phrases(bigramed_tokens, min_count=min_count, threshold=threshold, 
                    delimiter=b' ')
  trigram_phraser = Phraser(trigram)

  trigramed_tokens = []
  for sent in bigramed_tokens:
      tokens = trigram_phraser[sent]
      trigramed_tokens.append(tokens)

  # build a toy model to update with
  model = Word2Vec(size=300, min_count=5)
  model.build_vocab(trigramed_tokens)
  total_examples = model.corpus_count

  # add GloVe's vocabulary & weights
  model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

  # train on our data
  model.train(trigramed_tokens, total_examples=total_examples, epochs=model.epochs)
  model_wv = model.wv
  del model
  return model_wv

In [ ]:
for author in df['author'].unique():
  w2v_dict[author] = train_glove_author(author, glove_vectors=glove_vectors)
  print(f'{author} completed')

Plato completed
Aristotle completed
Locke completed
Hume completed
Berkeley completed
Spinoza completed
Leibniz completed
Descartes completed
Malebranche completed
Russell completed
Moore completed
Wittgenstein completed
Lewis completed
Quine completed
Popper completed
Kripke completed
Foucault completed
Derrida completed
Deleuze completed
Merleau-Ponty completed
Husserl completed
Heidegger completed
Kant completed
Fichte completed
Hegel completed
Marx completed
Lenin completed
Smith completed
Ricardo completed
Keynes completed


With this finished - our next step is to train one on the entire corpus for use in classification.

#### Building a Model for the full Corpus

In [ ]:
documents = df['gensim_tokenized']

# format the series to be used
stopwords = []

sentences = [sentence for sentence in documents]
cleaned = []
for sentence in sentences:
  cleaned_sentence = [word.lower() for word in sentence]
  cleaned_sentence = [word for word in sentence if word not in stopwords]
  cleaned.append(cleaned_sentence)

# get bigrams
bigram = Phrases(cleaned, min_count=30, threshold=10, 
                  delimiter=b' ')
bigram_phraser = Phraser(bigram)

bigramed_tokens = []
for sent in cleaned:
    tokens = bigram_phraser[sent]
    bigramed_tokens.append(tokens)

# run again to get trigrams
trigram = Phrases(bigramed_tokens, min_count=30, threshold=10, 
                  delimiter=b' ')
trigram_phraser = Phraser(trigram)

trigramed_tokens = []
for sent in bigramed_tokens:
    tokens = trigram_phraser[sent]
    trigramed_tokens.append(tokens)

# build a toy model to update with
all_text_model = Word2Vec(size=300, min_count=5)
all_text_model.build_vocab(trigramed_tokens)
total_examples = all_text_model.corpus_count

# add GloVe's vocabulary & weights
all_text_model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

# train on our data
all_text_model.train(trigramed_tokens, total_examples=total_examples, 
                     epochs=all_text_model.epochs)
all_text_wv = all_text_model.wv

As a test case, let's see how the philosophy thinks of itself as compared to how glove thinks of philosophy.

In [ ]:
for model in [1, 2]:
  if model == 1:
    print(f'\tPHILOSOPHY CORPUS')
    print('------------------------------------')
    test_w2v(all_text_wv, [(['philosophy'], [])])
  if model == 2:
    print(f'\tBASE GLOVE')
    print('------------------------------------')
    test_w2v(glove_vectors, [(['philosophy'], [])])


	PHILOSOPHY CORPUS
------------------------------------
Positive - ['philosophy']	Negative - []
- theology (0.8004)
- metaphysics (0.77228)
- religion (0.73747)
- science (0.72425)
- philosophical (0.71879)

	BASE GLOVE
------------------------------------
Positive - ['philosophy']	Negative - []
- theology (0.88151)
- philosophical (0.84362)
- mathematics (0.83389)
- psychology (0.82387)
- sociology (0.81085)



This sort of stands to reason - 'metaphysics' often has a different meaning outside of philosophical discussion, so it's not surprising to see it as the most changed term here. 

#### Finalized exporting

All in all, things look good, so let's export the vectors so that they can be used in our neural networks and in our dash app. 

In [ ]:
# do not run these cells if you want to keep old w2v versions
all_text_wv.save_word2vec_format('/gdrive/MyDrive/Colab_Projects/philosophy_data_project/w2v_models/w2v_for_nn.bin')
all_text_wv.save('/gdrive/MyDrive/Colab_Projects/philosophy_data_project/w2v_models/w2v_for_nn.wordvectors')

for source in w2v_dict.keys():
  w2v_dict[source].save(f'/gdrive/MyDrive/Colab_Projects/philosophy_data_project/w2v_models/{source}_w2v.wordvectors')

#### Exporting for Only New Texts


In [40]:
# use this cell to build the newest author/text/school in the corpus, then export in the cell below
epictetus_wv = train_glove(source_type='author', source='Epictetus', glove_vectors=glove_vectors)
stoicism_wv = train_glove(source_type='school', source='stoicism', glove_vectors=glove_vectors)

In [48]:
epictetus_wv.most_similar('good')

[('be', 0.9994500279426575),
 ('to', 0.9994434714317322),
 ('you', 0.9994379281997681),
 ('and', 0.9994164705276489),
 ('it', 0.9994126558303833),
 ('the', 0.999406099319458),
 ('that', 0.9993921518325806),
 ('is', 0.9993898868560791),
 ('he', 0.9993888139724731),
 ('of', 0.9993849396705627)]

In [41]:
epictetus_wv.save('/gdrive/MyDrive/Colab_Projects/philosophy_data_project/w2v_models/Epictetus_w2v.wordvectors')
stoicism_wv.save('/gdrive/MyDrive/Colab_Projects/philosophy_data_project/w2v_models/Stoiticism_w2v.wordvectors')

And that's it! See our other notebooks for more of the modeling work. 